# 2D classification task with FLaVor inference service

This guide will walk you through tailoring the FLaVor inference service for 2D classification tasks using ResNet18 inference model provided by `torchvision`.

## Prerequisite

As for the working environment, please ensure you have the following dependencies installed:

```
python >= 3.10
torch >= 1.13
torchvision >= 0.14
numpy < 2.0.0
```

or simply run:

In [ ]:
!poetry install --with cls_example

## Implementation

### Setup imports

In [ ]:
import os
from typing import Any, List, Sequence, Tuple

import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from torchvision.models import ResNet18_Weights, resnet18

from flavor.serve.apps import InferAPP
from flavor.serve.inference.data_models.api import (
    BaseAiCOCOImageInputDataModel,
    BaseAiCOCOImageOutputDataModel,
)
from flavor.serve.inference.data_models.functional import AiImage, InferCategory
from flavor.serve.inference.inference_models import BaseAiCOCOImageInferenceModel
from flavor.serve.inference.strategies import AiCOCOClassificationOutputStrategy

### Setup inference model

In this section, we would create `ClassificationInferenceModel` inheriting from `BaseAiCOCOImageInferenceModel`. There are few abstract methods that we must override such as `define_inference_network`, `set_categories`, `set_regressions`, `data_reader` and `output_formatter`. For the inference process related methods such as `preprocess`, `inference` and `postprocess`, we override them if necessary. `preprocess` and `postprocess` would remain an identical operation if unmodified. `inference` by default runs `self.forward(x)`.

Firstly, we need to implement submethods: `define_inference_network`, `set_categories` and `set_regressions`. These are defined in the `__init__()` constructor of the parent class `BaseAiCOCOImageInferenceModel`. `define_inference_network` defines your inference network and loads its pre-trained weight. `set_categories` and `set_regressions` define category and regression information. For example, a classification output would contain `c` channels. We need to show the exact meaning of each channel by specifying in `set_categories`. Refer to the following example for more detail.

Next, we implement other submethods that would be used in the `__call__` function of our inference model. See below workflow.

### `__call__` function workflow for the inference model
![__call__](images/call.png "inference workflow")

In [ ]:
class ClassificationInferenceModel(BaseAiCOCOImageInferenceModel):
    def __init__(self):
        self.formatter = AiCOCOClassificationOutputStrategy()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        super().__init__()

    def define_inference_network(self):
        network = resnet18(ResNet18_Weights.DEFAULT)
        network.eval()
        network.to(self.device)
        return network
    
    def set_categories(self):
        # ImageNet has 1000 categories
        categories = [
            {"name": str(i)} for i in range(1000)
        ]
        return categories

    def set_regressions(self):
        return None

    def data_reader(self, files: Sequence[str], **kwargs) -> Tuple[np.ndarray, None, None]:
        img = Image.open(files[0])
        return img, None, None

    def preprocess(self, data: np.ndarray) -> torch.Tensor:
        transforms = ResNet18_Weights.DEFAULT.transforms()
        img = transforms(data).unsqueeze(0).to(self.device)
        return img
    
    def inference(self, x: Any) -> Any:
        with torch.no_grad():
            out = self.network(x)
        return out

    def postprocess(self, model_out: torch.Tensor, **kwargs) -> np.ndarray:
        model_out = model_out.squeeze(0).cpu().detach()
        model_out = (nn.functional.softmax(model_out, dim=0) > 0.4).long()
        return model_out.numpy()

    def output_formatter(
        self,
        model_out: np.ndarray,
        images: Sequence[AiImage],
        categories: List[InferCategory],
        **kwargs
    ) -> Any:

        output = self.formatter(model_out=model_out, images=images, categories=categories)
        return output

### Integration with InferAPP
We could integrate our defined inference model with FLaVor `InferAPP`, a FastAPI application. To initiate the application, users have to define `input_data_model` and `output_data_model` which are the standard input and output structure for the service. Then, provide `infer_function` as the main inference operation. After initiate the service, `/invocations` API end point would be available to process the inference request. We encourge users to implement a stand-alone python script based on this jupyter notebook tutorial.

#### (Optional) to initiate application in jupyter notebook, you have to run the following block.


In [ ]:
# This block is only for jupyter notebook. You don't need this in stand-alone script.
import nest_asyncio
nest_asyncio.apply()

#### Initiate the service

In [ ]:
app = InferAPP(
    infer_function=ClassificationInferenceModel(),
    input_data_model=BaseAiCOCOImageInputDataModel,
    output_data_model=BaseAiCOCOImageOutputDataModel,
)

In [ ]:
app.run(port=int(os.getenv("PORT", 9111)))

### Send request
We can send request to the running server by `send_request.py` which opens the input files and the corresponding JSON file and would be sent via formdata. We expect to have response in AiCOCO format.

```bash
# pwd: examples/inference
python send_request.py -f test_data/cls_reg/n02123159_tiger_cat.jpeg -d test_data/cls_reg/input.json
```

## Setup Dockerfile
In order to interact with other services, we have to wrap the inference model into a docker container. Here's an example of the dockerfile.

```dockerfile
FROM nvidia/cuda:12.2.2-runtime-ubuntu20.04

RUN apt-get update \
    && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
        python3\
        python3-pip \
    && ln -sf /usr/bin/python3 /usr/bin/python

RUN pip install torch==2.1.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121 --default-timeout=1000
RUN pip install https://github.com/ailabstw/FLaVor/archive/refs/heads/release/stable.zip -U && pip install flavor

WORKDIR /app

COPY your_script.py  /app/

CMD ["python", "your_script.py"]

```